In [3]:
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16
from tqdm.notebook import tqdm
import os
import torch

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
def train_CNN(name,dataset_name):
    # Define data directory and image transforms
    data_dir = './Data/'
    # Create PyTorch datasets and dataloaders
    dataset = ImageFolder(os.path.join(data_dir, dataset_name), transform=data_transforms)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    # number of classes as number of directories in data_dir
    num_classes = len(os.listdir(data_dir + dataset_name))
    # Create VGG16 model with 5 neurons in last layer
    model = vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_features, num_classes)
    # make progress bar using tqdm
    # tqdm._instances.clear()
    
    # Train the model
    i=0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    model = model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(len(outputs))
            # print(len(labels))
            # print(inputs.shape)
            # print(outputs.shape)
            # print(labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            i+=1
            if i%100==0:
                print('Iteration loss: {:.4f}'.format(loss.item()))
        epoch_loss = running_loss / len(dataset)
        print('Epoch {}/{} loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))
    #save the model at path ../models/name.pth
    torch.save(model.state_dict(), '../models/'+name+'.pth')
    print('Successfully trained ' + name + '!' )


In [6]:
#code if you wish to load model named 'name' from path '../models/name.pth'
def load_model_name(name):
    model1 = vgg16(pretrained=True)
    for param in model1.parameters():
        param.requires_grad = False
    num_features = model1.classifier[6].in_features
    model1.classifier[6] = nn.Linear(num_features, 5)
    model1.load_state_dict(torch.load('../models/'+name+'.pth'))
    model1.eval()
    return model1

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [8]:
#code to rotate images by 180 degrees in a directory

# from PIL import Image,ImageOps
# import os

# # Define the directory containing the images
# directory = './data/Face_Inverted'

# # Loop through all subdirectories and files in the directory
# for root, directories, files in os.walk(directory):
#     for file in files:
#         # Check if the file is an image
#         if file.endswith(('png', 'jpg', 'jpeg')):
#             # Open the image and invert it
#             image_path = os.path.join(root, file)
#             image = Image.open(image_path)
#             rotated_image = image.rotate(180)
#             # Save the rotated image
#             rotated_image.save(image_path)



In [12]:
#Gotta train these four:

# train_CNN('Face_ID_Inverted_CNN','Face_Inverted')
# train_CNN('Asian_Face_ID_CNN','Asian_Faces')
# train_CNN('White_Face_ID_CNN','White_Faces')
# train_CNN('Face_ID_CNN','Face_Inverted_original')


